Decoratori
==========

La lezione è basata su https://realpython.com/primer-on-python-decorators/.

Un decoratore è una funzione che prende come argomento un'altra funzione e ne amplia il comportamento senza modificarla eplicitamente. 

In Python le funzioni sono **Oggetti di Prima Classe** cioè possono essere passate come argomenti ad altre funzioni come qualunque altro oggetto (int, float, list etc.)

In [12]:
def say_hello(name):
    return f"Hello {name}"


def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"


def greet_bob(greeter_func, name ="Bob"):
    return greeter_func(name)

In [13]:
greet_bob(say_hello) # function name as argument

'Hello Bob'

In [4]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

In [15]:
greet_bob(say_hello,"Peter") # function name and its input as arguments

'Hello Peter'

Possiamo definire funzioni all'interno di funzioni e una funzione può restituire funzioni:

In [5]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

`parent` restituisce una referenza (indirizzo di memoria) a una funzione (Si noti che nel `return` il nome della funzione è privo di parentesi tonde. Provate e restituire `first_child()` ), che possiamo poi invocare normalmente:

In [8]:
first = parent(1)
first

<function __main__.parent.<locals>.first_child()>

In [9]:
first()

'Hi, I am Emma'

### Un primo esempio di decoratore

In [1]:
def my_decorator(func):   # decorator on a generic function with no input
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")

    return wrapper    # return reference to an internal function. `wrapper` non può essere invocato dall'esterno di `my_decorator`

In [2]:
wrapper()

NameError: name 'wrapper' is not defined

Definiamo una funzione:

In [3]:
def say_whee():
    print("Whee!")

Definiamo una versione decorata di `say_whee`:

In [4]:
decorated_say_whee = my_decorator(say_whee)  # a wrapper viene pasata la referenza alla funzione `say_whee`

In [5]:
decorated_say_whee()   # decorated_say_whee viene effettivamente eseguita. `func` in wrapper ora punta a `say_whee`.

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [6]:
print(say_whee)
print(decorated_say_whee)

<function say_whee at 0x10d31c0d0>
<function my_decorator.<locals>.wrapper at 0x10d31c160>


Possiamo anche ridefinire `say_whee` come la versione decorata di se stessa:

In [8]:
say_whee = my_decorator(say_whee)

In [9]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [10]:
print(say_whee)

<function my_decorator.<locals>.wrapper at 0x10d31c430>


Riassumendo: un decoratore si `avvolge` attorno ad una funzione e ne modifica il comportamento.

### Esempio 2

L'azione svolta da un decoratore può essere determinato dinamicamente, cioè quando la funzione decorata viene chiamata

In [26]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep.

    return wrapper

In [28]:
def honk():
    print("Honk! Honk!")

In [29]:
honk = not_during_the_night(honk)

In [30]:
honk()

Honk! Honk!


Se chiamate `honk` fra le 22 e le 7 non succede nulla.

Esiste un modo sintetico di ridefinire una funzione come la funzione stessa inglobata in un decoratore, usando il simbolo `@`:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")

    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

`@my_decorator` prima della definizione di `say_whee` è equivalente a definire say_whee e poi aggiugere la linea 
`say_whee = my_decorator(say_whee)`.

### Riutilizzare i decoratori

Se i decoratori vengono salvati in un file possono essere importati come normali funzioni e utilizzati in ambiti diversi.

Esempio:

Supponiamo che in decoratori.py ci sia il codice che segue:
```
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice
```

In [32]:
from decorators import do_twice

@do_twice       # declare the decorated function
def say_whee_twice():
    print("Whee!")

In [33]:
say_whee_twice()  # call the decorated function

Whee!
Whee!


### Decorare funzioni con argomenti
Per decorare funzioni con input arbitrari è necessario utilizzare la notazione `*args, **kwargs`. L'input è passato al wrapper interno che a sua volta lo passa alle funzioni da decorare.

In [34]:
def do_twice(func):
    """Run the decorated function twice"""
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [35]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [36]:
greet("Pippo")

Hello Pippo
Hello Pippo


### Ritornare valori da una funzione decorata
Nell'esempio seguente l'introduzione del decoratore functools.wraps serve a facilitare l'accesso alla funzione più interna

In [41]:
import functools

def do_twice(func):
    """Run the decorated function twice"""
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [42]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [39]:
return_greeting("Paperino")

Creating greeting
Creating greeting


'Hi Paperino'

Verifichiamo che la funzione decorata ritorni effettivamente il risultato:

In [40]:
print("La funzione ha restituito:",return_greeting("Paperino"))

Creating greeting
Creating greeting
La funzione ha restituito: Hi Paperino


### Decoratori nella definizione di classi

Python possiede dei decoratori built-in che vengono comunemente utilizzati nella costruzione di classi, @classmethod, @staticmethod, and @property. I decoratori @classmethod and @staticmethod vengono usati per definire metodi interni alla classe che non sono collegati alle particolari istanze della classe stessa. Il decoratore @property è utilizzato per personalizzare i getters and setters degli attributi della classe.

In [59]:
# I `@classmethods` sono essenzialmente degli inizializzatori più specializzati
# Notare che @classmethods ha come primo argomento il nome della classe e che viene ereditato dalle sottoclassi
# L'underscore in self._radius = radius vuol dire che radius è un attributo PRIVATO
# @property introduce un attributo (può essere richiamato senza le parentesi () )
# In mancanza di un metodo `setter` non può essere modificato dall'esterno
class Circle:
    
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [60]:
C1 = Circle(3)

In [61]:
C1.radius

3

In [62]:
C1.area

28.2743338815

In [63]:
C1.radius = 3.4

In [64]:
C1.area

36.31681107446

In [65]:
C1.area = 37.9

AttributeError: can't set attribute